# 0 环境配置

In [1]:
basedir = "/home/ubuntu/dishui_ddd/input/"

In [2]:
import os
import cv2
import glob
import numpy as np
import pandas as pd

from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.applications import *
from keras.preprocessing.image import *
print("import done")

Using TensorFlow backend.


import done


# 1 预设参数

In [3]:
modelname = "Model2_ResNet50_DG1_L109"

In [4]:
from keras.applications.resnet50 import preprocess_input

In [5]:
# changing
model_image_size = (240, 320)
fine_tune_layer = 106
batch_size = 64

# 2 数据生成

In [6]:
train_gen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=10.,
    width_shift_range=0.05,
    height_shift_range=0.05,
    shear_range=0.1,
    zoom_range=0.1,
)

gen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
)

train_generator = train_gen.flow_from_directory(os.path.join(basedir, 'train'),  model_image_size, \
                                                shuffle=True, batch_size=batch_size, class_mode="categorical")

valid_generator = gen.flow_from_directory(os.path.join(basedir, 'valid'),  model_image_size, \
                                          shuffle=True, batch_size=batch_size, class_mode="categorical")

test_generator = gen.flow_from_directory(os.path.join(basedir,"test"),  model_image_size, \
                                         shuffle=False, batch_size=batch_size, class_mode=None)

Found 20062 images belonging to 10 classes.
Found 2362 images belonging to 10 classes.
Found 79726 images belonging to 1 classes.


# 3 搭建模型

In [21]:
from keras import backend as K
K.clear_session()

In [22]:
inputs = Input((*model_image_size, 3))
base_model = ResNet50(input_tensor=inputs, weights='imagenet', include_top=False)
x = GlobalAveragePooling2D()(base_model.output)
x = Dropout(0.5)(x)
x = Dense(10, activation='softmax')(x)
model = Model(inputs, x)

print("total layer count {}".format(len(base_model.layers)))

for i in range(fine_tune_layer):
    model.layers[i].trainable = False

total layer count 174


# 4 训练模型

In [8]:
steps_train_sample = int(train_generator.samples*0.75)//batch_size + 1
steps_valid_sample = int(valid_generator.samples*0.75)//batch_size + 1
print("train_generator.samples = {}".format(steps_train_sample))
print("valid_generator.samples = {}".format(steps_valid_sample))

train_generator.samples = 236
valid_generator.samples = 28


## （1） epoch 1，学习率1e-4，使用Adam

In [23]:
model.compile(optimizer=Adam(lr=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit_generator(train_generator, steps_per_epoch=steps_train_sample, epochs=1, \
                    validation_data=valid_generator, validation_steps=steps_valid_sample)

Epoch 1/1
236/236 [==============================] - 366s - loss: 0.3838 - acc: 0.8785 - val_loss: 0.3923 - val_acc: 0.8901


In [24]:
model.save("models/"+modelname+".h5")
print("model saved!")

model saved!


In [32]:
model = load_model("models/"+modelname+".h5")

## （2） epoch 2，学习率1e-6，使用RMSprop

In [27]:
model.compile(optimizer=RMSprop(lr=1e-6), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit_generator(train_generator, steps_per_epoch=steps_train_sample, epochs=1, \
                     validation_data=valid_generator, validation_steps=steps_valid_sample)

Epoch 1/1
236/236 [==============================] - 364s - loss: 0.0476 - acc: 0.9849 - val_loss: 0.3710 - val_acc: 0.8956


In [28]:
model.save("models/"+modelname+".h5")
print("model saved!")

model saved!


# 5 输出测试

In [33]:
def predict_test(model,  model_image_size, csv_name):

    y_pred = model.predict_generator(test_generator,  steps=test_generator.samples//batch_size+1,  verbose=1)

    l = list()
    for i, fname in enumerate(test_generator.filenames):
        name = fname[fname.rfind('/')+1:]
        l.append( [name, *y_pred[i]] )

    l = np.array(l)
    data = {'img': l[:,0]}
    for i in range(10):
        data["c%d"%i] = l[:,i+1]
    df = pd.DataFrame(data, columns=['img'] + ['c%d'%i for i in range(10)])
    df = df.sort_values(by='img')
    df.to_csv(csv_name, index=None, float_format='%.3f')
    print("submission saved")

In [34]:
predict_test(model,  model_image_size, "submission/"+modelname+".csv")

1246/1246 [==============================] - 1877s  
submission saved
